In [1]:
import pandas as pd

In [2]:
df_ldap = pd.read_pickle('../ldap/ldap.pkl')

In [3]:
df_ldap.head(5)

,employee_name,user_id,email,role,business_unit,functional_unit,department,team,supervisor,date
0,Macey Colleen Nash,MCN0973,Macey.Colleen.Nash@dtaa.com,ElectricalEngineer,1,3 - Manufacturing,2 - ManufacturingEngineering,NaN,Ann Hannah Dickerson,2009-12
1,Nelle Sharon Cherry,NSC0622,Nelle.Sharon.Cherry@dtaa.com,Technician,1,5 - SalesAndMarketing,3 - FieldService,3 - RegionalFieldService,Amethyst Suki Norris,2009-12
2,Kathleen Audrey Vargas,KAV0428,Kathleen.Audrey.Vargas@dtaa.com,ProductionLineWorker,1,3 - Manufacturing,3 - Assembly,2 - AssemblyDept,Brenna Martha Russell,2009-12
3,Adrienne Joelle Mclean,AJM0772,Adrienne.Joelle.Mclean@dtaa.com,Scientist,1,2 - ResearchAndEngineering,1 - Research,5 - Lab,Emmanuel Casey Wiggins,2009-12
4,Nicole Maris Valentine,NMV0507,Nicole.Maris.Valentine@dtaa.com,Manager,1,5 - SalesAndMarketing,2 - Sales,3 - RegionalSales,Perry Reese Morton,2009-12


In [4]:
df_users = pd.DataFrame(df_ldap['user_id'].unique())

In [5]:
df_dates = pd.date_range('1/1/2010', '5/31/2011').to_frame()
df_dates[0] = df_dates[0].dt.strftime('%Y%m%d')

In [6]:
df_master = pd.merge(df_users.assign(key=1), df_dates.assign(key=1), on='key').drop('key', axis=1)

In [63]:
df_master

,user_id,dateString,userDate,isEmployed,isInsider,hasLogin,firstActivity,hasConnect,role,dateHour,...,psyc_C,psyc_E,psyc_A,psyc_N,numMonthsEmployed,sumExtEmailSize,sumExtEmailAttachments,numExtEmails,probLogonUser,changeNumRole
0,MCN0973,20100101,MCN0973_20100101,1,0,NaN,NaN,NaN,ElectricalEngineer,NaN,...,39,41,14,29,18,NaN,NaN,NaN,0.000000,0
1,MCN0973,20100102,MCN0973_20100102,1,0,NaN,NaN,NaN,ElectricalEngineer,NaN,...,39,41,14,29,18,NaN,NaN,NaN,0.000000,0
2,MCN0973,20100103,MCN0973_20100103,1,0,NaN,NaN,NaN,ElectricalEngineer,NaN,...,39,41,14,29,18,NaN,NaN,NaN,0.000000,0
3,MCN0973,20100104,MCN0973_20100104,1,0,1.0,NaN,NaN,ElectricalEngineer,9.0,...,39,41,14,29,18,21124.0,0.0,1.0,0.398942,0
4,MCN0973,20100105,MCN0973_20100105,1,0,1.0,NaN,NaN,ElectricalEngineer,9.0,...,39,41,14,29,18,NaN,NaN,NaN,0.398942,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642975,FPS0900,20110527,FPS0900_20110527,1,0,NaN,NaN,NaN,ComputerProgrammer,NaN,...,48,39,16,30,18,NaN,NaN,NaN,0.000000,1
642976,FPS0900,20110528,FPS0900_20110528,1,0,NaN,NaN,NaN,ComputerProgrammer,NaN,...,48,39,16,30,18,NaN,NaN,NaN,0.000000,1
642977,FPS0900,20110529,FPS0900_20110529,1,0,NaN,NaN,NaN,ComputerProgrammer,NaN,...,48,39,16,30,18,NaN,NaN,NaN,0.000000,1
642978,FPS0900,20110530,FPS0900_20110530,1,0,NaN,NaN,NaN,ComputerProgrammer,NaN,...,48,39,16,30,18,NaN,NaN,NaN,0.000000,1


In [8]:
df_master.rename(columns={'0_y':'dateString','0_x':'user_id'}, inplace=True)

In [9]:
df_master['userDate'] = df_master.apply(lambda x: x['user_id'] + '_' + x['dateString'], axis = 1)

In [10]:
endDates = df_ldap.groupby('user_id')['date'].max()

In [11]:
endDates['WCC0009']

'2011-04'

In [12]:
'20100101'[0:6]

'201001'

In [13]:
def isEmployed(row):
    user = row['user_id']
    dateString = int(row['dateString'][0:6])
    # It is a little of a hack, but comparing -1 from 201012 201100 is still ok.
    minusMonth = dateString - 1
    if int(endDates[row['user_id']].replace('-','')) >= minusMonth:
        return 1
    else:
        return 0 
    

In [14]:
df_master['isEmployed'] = df_master.apply(isEmployed, axis=1)

In [15]:
df_master[(df_master['user_id'] == 'CCH0959') & (df_master['dateString'] == '20100901')]

,user_id,dateString,userDate,isEmployed
434199,CCH0959,20100901,CCH0959_20100901,1


In [16]:
insiders = {'CSF0929':['20100701','20100702','20100703','20100708','20100709','20100714','20100716'], 'CCH0959':['20100802','20100803','20100804','20100805','20100806','20100809',
                                                                                                                 '20100810','20100811','20100812','20100813','20100816','20100817','20100818',
                                                                                                                 '20100819','20100820','20100823','20100824','20100825','20100826','20100827',
                                                                                                                 '20100830','20100831','20100901','20100902','20100903','20100907','20100908',
                                                                                                                 '20100909','20100910','20100910','20100913','20100914','20100915','20100916',
                                                                                                                 '20100917','20100920','20100921','20100922','20100923','20100924','20100927',
                                                                                                                 '20100928','20100929','20100930']}

In [17]:
def isInsider(row):
    if row['user_id'] in insiders.keys() and row['dateString'] in insiders[row['user_id']]:
        return 1
    else:
        return 0

In [18]:
df_master['isInsider'] = df_master.apply(isInsider, axis=1)

In [19]:
df_master[(df_master['user_id'] == 'CSF0929') & (df_master['dateString'] == '20100801')]

,user_id,dateString,userDate,isEmployed,isInsider
44588,CSF0929,20100801,CSF0929_20100801,0,0


In [20]:
df_master.columns

Index(['user_id', 'dateString', 'userDate', 'isEmployed', 'isInsider'], dtype='object')

In [21]:
#df_master[df_master['hasLogin'] > 0][['userDate','user_id','dateHour','probLogonUser']].head(50)
df_master['probLogonUser'].unique()

KeyError: 'probLogonUser'

In [22]:
df_hasLogin = pd.read_pickle('../logon/hasLogins.pkl')

df_master = pd.merge(df_master, df_hasLogin, how='left', on='userDate')

df_master.rename(columns={'count':'hasLogin'}, inplace=True)

In [23]:
df_firstActivity = pd.read_pickle('../device/firstActivity.pkl')

df_master = pd.merge(df_master, df_firstActivity[df_firstActivity['activity'] == 'Connect'], how='left', on='userDate')

df_master.drop('activity', axis = 1, inplace=True)

In [24]:
df_hasConnect = pd.read_pickle('../device/hasConnect.pkl')

df_master = pd.merge(df_master, df_hasConnect, how='left', on='userDate')

In [25]:
df_userRole = pd.read_pickle('../ldap/userRole.pkl')

df_master = pd.merge(df_master, df_userRole, how='left', on='user_id')

In [27]:
#this will create duplicate userDate columns
df_logonHour = pd.read_pickle('../logon/logonHour.pkl')

df_master = pd.merge(df_master, df_logonHour, how='left', on='userDate')

In [30]:
df_roleProbHour = pd.read_pickle('../logon/roleProbHour.pkl')

def probHour(row):
    if row['hasLogin'] > 0:
        roleProbHour = df_roleProbHour.loc[row['role'], row['dateHour']]
    else:
        roleProbHour = 0
    
    return roleProbHour


df_master['probLogonRole'] = df_master.apply(probHour, axis=1)

In [32]:
df_urlByuserDate = pd.read_pickle('../http/urlByUserDate.pkl')
df_master = pd.merge(df_master, df_urlByuserDate.rename(columns={'count':'webCount'}), how='left', on='userDate')
# test for weirdness : df_temp[(df_temp['hasLogin'] == 0) & (df_temp['webCount'] > 0)]
# df_temp[(df_temp['hasLogin'] > 0) & (df_temp['webCount'] > 0)]
# df_temp[(df_temp['hasLogin'] > 0) & (df_temp['webCount'] > 0)]


20230304 - added questionable usage

In [34]:
df_webQuestionsableUsage = pd.read_pickle('../http/insiderWebCount.pkl')
df_master = pd.merge(df_master, df_webQuestionsableUsage.rename(columns={'isInsiderWeb':'webQuestionableUsage'}), how='left', on='userDate')
# df_master[(df_master['hasLogin'] > 0) & (df_master['webCount'] > 0)]

20230224 - added numPC and psychometric

In [36]:
df_numPC = pd.read_pickle('../logon/numPC.pkl').reset_index()
df_master = pd.merge(df_master, df_numPC, how='left', on='userDate')
#df_test[(df_test['countPC'] > 1) & ~(np.isnan(df_test['hasLogin']))] 
# may dup hasLogin

In [38]:
df_psychometic = pd.read_pickle('../psychometric/psychometic.pkl')
df_master = pd.merge(df_master, df_psychometic, how='left', on='user_id')

In [40]:
df_numMonthsEmployed = pd.read_pickle('../ldap/numMonthsEmployed.pkl')
df_numMonthsEmployed = df_numMonthsEmployed.to_frame().reset_index().rename(columns={'date':'numMonthsEmployed'})
#df_numMonthsEmployed[df_numMonthsEmployed['numMonthsEmployed'] != 18]
df_master = pd.merge(df_master, df_numMonthsEmployed, how='left', on='user_id')

In [41]:
df_emailCounts = pd.read_pickle('../email/emailCounts.pkl')
df_master = pd.merge(df_master, df_emailCounts, how='left', on='userDate')

In [42]:
df_userProbHour = pd.read_pickle('../logon/userProbHour.pkl')

def probUser(row):
    if row['hasLogin'] > 0:
        probLogonUser = df_userProbHour.loc[row['user_id'], row['dateHour']]
    else:
        probLogonUser = 0
    
    return probLogonUser

df_master['probLogonUser'] = df_master.apply(probUser, axis=1)

In [58]:
df_changeRoles = pd.read_pickle('../ldap/changeRoles.pkl')
df_changeRoles = df_changeRoles.set_index('dateString')

def changeInRoles(row):
    changeInRole = df_changeRoles.loc[row['dateString'],row['role']]
    return changeInRole

df_master['changeNumRole'] = df_master.apply(changeInRoles, axis=1)
# test : df_master[(df_master['changeNumRole'] > 0 ) & (df_master['role'] == "AdministrativeAssistant")]

In [ ]:
#not longer needed - note on num logins
#df_master.drop_duplicates(inplace=True)

In [64]:
df_master.to_parquet('./master.parquet')

In [44]:
df_master = pd.read_parquet('./master.parquet')

FileNotFoundError: [Errno 2] No such file or directory: './master.parquet'

In [46]:
df_master.columns

Index(['user_id', 'dateString', 'userDate', 'isEmployed', 'isInsider',
       'hasLogin', 'firstActivity', 'hasConnect', 'role', 'dateHour',
       'probLogonRole', 'webCount', 'webQuestionableUsage', 'countPC',
       'employee_name', 'psyc_O', 'psyc_C', 'psyc_E', 'psyc_A', 'psyc_N',
       'numMonthsEmployed', 'sumExtEmailSize', 'sumExtEmailAttachments',
       'numExtEmails', 'probLogonUser'],
      dtype='object')

In [ ]:
# no longer needed switch to parquet
#df_master.to_pickle('./master.pkl')

In [ ]:
# no longer needed switch to parquet
#df_master = pd.read_pickle('./master.pkl')

In [47]:
df_master[['dateString', 'userDate']]

,dateString,userDate
0,20100101,MCN0973_20100101
1,20100102,MCN0973_20100102
2,20100103,MCN0973_20100103
3,20100104,MCN0973_20100104
4,20100105,MCN0973_20100105
...,...,...
642975,20110527,FPS0900_20110527
642976,20110528,FPS0900_20110528
642977,20110529,FPS0900_20110529
642978,20110530,FPS0900_20110530
